# Term Paper Project: EU Transparency Register

Research Question: Which lobby groups have been in contanct with members of the European Commission (EC) or the Directorates-Generale (DG) ?

In [1]:
# load packages
import pandas as pd
import numpy as np
import os

# Load & Transform Data
- Load Data from Transparency register
- filter for Meetings within the relevant time period

In [2]:
# load data

meetings_ec = pd.read_excel("./original_data/Meetings of Commission representatives of the Von der Leyen Commission (2019-2024).xlsx", header=1)
meetings_df = pd.read_excel("./original_data/Meetings of Directors-General of the European Commission.xlsx", header=1)

# transform subject column to lower case to improve keyword search
meetings_ec["Subject of the meeting"] = meetings_ec["Subject of the meeting"].str.lower()
meetings_df["Subject of the meeting"] = meetings_df["Subject of the meeting"].str.lower()

# filter for entries between 
min_date = "2020-06-01"
max_date = "2023-04-18"

meetings_df['Date of meeting'] = pd.to_datetime(meetings_df['Date of meeting'], format="%Y-%m-%d")
meetings_ec['Date of meeting'] = pd.to_datetime(meetings_ec['Date of meeting'], format="%Y-%m-%d")


meetings_df = meetings_df.loc[(meetings_df['Date of meeting'] >= min_date)
                     & (meetings_df['Date of meeting'] <= max_date)]

meetings_ec = meetings_ec.loc[(meetings_ec['Date of meeting'] >= min_date)
                     & (meetings_ec['Date of meeting'] <= max_date)]



In [3]:
# select all relevant meetings by filtering for keywords in the column "Subject of the meeting"

chips_act_keywords = ["chips", "semiconduct", "chip", "2023/1781", "microelectronic", "foundry", "advanced manufacturing", "ipcei", "node", "semion", "wafer"]
keywords_to_exclude = ["schiphol", "mobility ipcei", "ipcei, hydrogen", "ipcei electrolyzer", 
                       "discussion on ipceis", "h2 ipcei", "snam would like to exchange on the work on the ipcei", 
                       "battery ipceis", "energy markets and ipcei", "overall discussion on energy matters, ipcei, temporary framework"]

chips_meetings_ec = meetings_ec[
    (meetings_ec["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True))  
    & (meetings_ec["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
    ].copy()
chips_meetings_dg = meetings_df[
    (meetings_df["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True)) 
    & (meetings_df["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
    ].copy()

# Merge DG and EC dataframe

# rename columns, drop interest group name column (group names are retrieved from register later), create col to indicate whether meeting took place with EC or DG rep.
chips_meetings_dg.rename(columns={"Name of DG - full name": "Name of DG / EC cabinet"}, inplace=True)
chips_meetings_ec.rename(columns={"Name of cabinet": "Name of DG / EC cabinet"}, inplace=True)
chips_meetings_dg.loc[:, "EC_or_DG"] = "DG"
chips_meetings_ec.loc[:, "EC_or_DG"] = "EC"


chips_meetings = pd.concat([chips_meetings_ec, chips_meetings_dg])
chips_meetings.drop(columns=["Name of interest representative"], inplace=True)



In [4]:
# load transparency register

transparency_register = pd.DataFrame(columns=["Transparency register ID", "Head Office"])

# Head office country, Identification number:


for entry in os.scandir("./original_data"):  
    if entry.is_file() and entry.name.endswith(".xls"):  # check if it's a xlsx file
        print(entry.path)
        xls = pd.read_excel(entry.path)

        if "Identification number:" in xls.columns:
            xls = xls[["Head office country", "Identification number:", "(Organisation) name"]] 
            xls.rename(columns={"Identification number:": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification number" in xls.columns:
            xls = xls[["Head office country", "Identification number", "(Organisation) name"]]
            xls.rename(columns={"Identification number": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification code" in xls.columns:
            xls = xls[["Head office country", "Identification code", "(Organisation) name"]]
            xls.rename(columns={"Identification code": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        else:
            raise KeyError("ERROR MISSING TRANSPARENCY REGISTER ID COLUMN!")
        transparency_register = pd.concat([transparency_register, xls]).drop_duplicates(subset="Transparency register ID")

transparency_register["Head Office"] = transparency_register["Head Office"].str.upper()
transparency_register.rename(columns={"(Organisation) name": "Interest Group Name"}, inplace=True)
# transparency_register.to_excel("./original_data/merged_register.xlsx", index=False)



./original_data\Organisations in Transparency Register - 2019 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2019 - JUNE.xls
./original_data\Organisations in Transparency Register - 2020 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2020 - JUNE.xls
./original_data\Organisations in Transparency Register - 2021 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2021 - JUNE.xls
./original_data\Organisations in Transparency Register - 2022 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2022 - JUNE.xls
./original_data\Organisations in Transparency Register - 2023 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2023 - JUNE.xls
./original_data\Organisations in Transparency Register - 2024 - JAN.xls


In [5]:
# Handle Meetings with multiple entries in Register ID Column: Create as many rows as there are interests groups participating

# iterate through rows with multiple REG in the same cell
new_rows = pd.DataFrame(columns=chips_meetings.columns)

for index, row in chips_meetings[chips_meetings["Transparency register ID"].str.contains(",")].iterrows():
    reg_ids = str(row["Transparency register ID"]).split(",")
    reg_id_rows = pd.DataFrame(columns=new_rows.columns)
    
    for id in reg_ids:
        row["Transparency register ID"] = id
        reg_id_rows = pd.concat([reg_id_rows, row.to_frame().transpose()])
    
    new_rows = pd.concat([new_rows, reg_id_rows])

chips_meetings_appended = pd.concat([chips_meetings[chips_meetings["Transparency register ID"].str.contains(",") == False], new_rows])

meetings_df = pd.merge(left=chips_meetings_appended, right=transparency_register, how="left", on="Transparency register ID")

# reorder columns
meetings_df = meetings_df.iloc[:, [7, 0, 1, 8, 2, 10, 5, 9, 6, 3, 4]]


In [6]:
# export all interest groups to xlsx for categorization and name cleaning

interestgroups_names = meetings_df[["Transparency register ID", "Interest Group Name"]].drop_duplicates()
interestgroups_names.to_excel("./manually_edited_data/interest_groups_categories.xlsx", index=False)

## Add Region column

In [ ]:
mapping = {
    "AUSTRIA": "EUROPE",
    "BELGIUM": "EUROPE",
    "DENMARK": "EUROPE",
    "FRANCE": "EUROPE",
    "GERMANY": "EUROPE",
    "ITALY": "EUROPE",
    "NETHERLANDS": "EUROPE",
    "NORWAY": "EUROPE (NON-EU)",
    "SPAIN": "EUROPE",
    "SWEDEN": "EUROPE",
    "TAIWAN": "ASIA",
    "UNITED KINGDOM": "EUROPE (NON-EU)",
    "UNITED STATES": "USA"
}

meetings_df["Region"] = meetings_df["Head Office"].map(mapping)

## Read manually edited data and merge into main dataframe "meeting_df"

In [8]:
categorized_df = pd.read_excel("./manually_edited_data/interest_groups_categories_result.xlsx", sheet_name="data")
categorized_df = categorized_df[["Transparency register ID", "Interest Group Name", "Category Broad", "Category Detailed"]]
final_data = meetings_df.merge(right=categorized_df, 
                                how="left", 
                                on=["Transparency register ID", "Interest Group Name"])

final_data.to_excel("./final_data/final_meetings_data.xlsx", index = False)

d:\Development\Python Data Science\Transparency Project\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
